In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import eig

# 2019-09-18-TF_kinetics

Transcription factor kinetics on different topological manifolds can be calculated with some nice theoretical consideration.

## FPT when they escape
Let's consider a network with a given structure. I have previously characterized the FPT, MFPT, and GMFPT behaviour of random walks on a network using various theoretical approaches. Now I want to consider a situation in which I add a node to the network, where this node corresponds to a "void" state (bulk diffusion), which has a link to all other nodes, with a given weight.

In [ ]:
def GMFPT_theory (A,weighted=True) :
    """
    According to the theory of Lin et al., 2012, the global mean first passage
    time can be calculated by finding the eigenspectrum of the Laplacian matrix
    of the graph. This function calculates the GMFPT from their formula, for the
    graph described by the adjacency matrix A, to all sites. Optional parameter
    'weighted' allows for the choice of having the same quantity but weighted
    with the stationary distribution.
    """
    N = A.shape[0]
    d = np.sum(A,axis=1)
    E = np.sum(d)/2.
    L = np.diag(d) - A
    L_eigs = eig(L)
    sortidx = np.argsort(L_eigs[0])
    l = np.array([L_eigs[0][i].real for i in sortidx])
    v = np.array([L_eigs[1][:,i].real for i in sortidx])
    T = np.zeros(N)
    dv = np.dot (v,d)
    if not weighted :
        for j in range(N) :
            for i in range(1,N) :
                T[j] += 1.0/l[i] * (2*E*v[i,j]**2 - v[i,j]*dv[i])
        return float(N)/(N-1.0) * T
    else :
        for j in range(N) :
            for i in range(1,N) :
                dvi = v[i,j]*dv[i]
                T[j] += 1.0/l[i]*((2*E)**2*v[i,j]**2 - 2*v[i,j]*2*E*dvi - dvi**2)
        return T/(2*E)

In [ ]:
def matrix_with_void(mat0, p_void) :
    N = mat0.shape[0]
    d_j = np.sum(mat0,axis=1)
    lambda_j = p_void * d_j / (1-p_void)
    
    # init the new matrix and fill it with the previous elements
    mat = np.zeros((N+1,N+1))
    mat[:N,:N] = mat0
    
    # put the extra state
    mat[N,:-1] = lambda_j
    mat[:-1,N] = lambda_j
    
    return mat

In [ ]:
def uniform_decay(n, alpha) :
    # init matrix
    m = np.zeros((n,n))

    # init p(s) function
    ps = np.arange(1,n)**alpha
    ps /= ps.sum()

    # for i in 
    for i in range(n) :
        m[i,i+1:] = ps[:n-i-1]
        m[i,:i] = np.flip(ps[:i])
    
    return m

In [ ]:
def add_tad(mat, tad_start, tad_end, tad_alpha) :
    n = tad_end - tad_start
    mat[tad_start:tad_end, tad_start:tad_end] += uniform_decay(n, tad_alpha)
    return mat

In [ ]:
def add_noise(M, n_noise) :
    N = M.shape[0]
    coords = np.random.randint(low=0, high=N, size=(n_noise,2))
    for i,j in coords :
        M[i,j] += 1

## Chain
Let's start with a simple example: a chain.

In [ ]:
N = 10
p_void = 0.3

# init the chain
chain0 = np.zeros((N,N))
for i in range(1,N) :
    chain0[i,i-1] = 1.0
    chain0[i-1,i] = 1.0
    
# now I want to add a new node to the network. I create a new adjacency
# matrix
chain = matrix_with_void(chain0, p_void)

In [ ]:
plt.matshow(chain)

In [ ]:
# calculate the GMFPT as a function of the site
chain_gmfpt = GMFPT_theory(chain)

In [ ]:
# plot
plt.plot(chain_gmfpt[:N],linewidth=3)
plt.xlabel("Site index",fontsize=18)
plt.ylabel("GMFPT",fontsize=18)
plt.title (r"Chain, $p_{void}$ = %.1e"%p_void)
plt.show()

In [ ]:
N = 1000
alpha = -1.0

B0 = uniform_decay(N, alpha)
B = matrix_with_void(B0, p_void)

In [ ]:
plt.matshow(np.log(B), cmap=plt.cm.Greys)

In [ ]:
B_gmfpt = GMFPT_theory(B)
plt.plot(B_gmfpt[:N],linewidth=3)
plt.xlabel("Site index",fontsize=18)
plt.ylabel("GMFPT",fontsize=18)
plt.title (r"B, $p_{void}$ = %.1e"%p_void)
plt.show()

In [ ]:
tad.sum()

In [ ]:
tad0 = uniform_decay(N, alpha)
tad0 = add_tad(tad0, 0, 300, -0.5)
tad0 = add_tad(tad0, 200, 500, -0.5)
tad0 = add_tad(tad0, 400, 600, -0.5)
tad0 = add_tad(tad0, 550, 900, -0.5)
tad0 = add_tad(tad0, 850, 1000, -0.5)
tad = matrix_with_void(tad0, p_void)

# now give it a more "realistic" touch
n_reads = 10000
noise_ratio = 0.9
n_noise = int(n_reads * noise_ratio)
tad *= n_reads
add_noise(tad, n_noise)

plt.matshow(np.log(tad))

In [ ]:
tad

In [ ]:
tad_gmfpt = GMFPT_theory(tad)
plt.plot(tad_gmfpt[:N]/tad_gmfpt[:N].min(), linewidth=3)
plt.xlabel("Site index",fontsize=18)
plt.ylabel("GMFPT",fontsize=18)
plt.title (r"B, $p_{void}$ = %.1e"%p_void)
plt.show()